In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip your_path_to_the_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access pub

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

360

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1324

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_to

 LLaMA-2 was trained on 2 trillion tokens.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
 LLaMA-2 was trained on 2 trillion tokens.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip your_path_to_the_zip_file
!unzip -q Questionscsv.zip -d questions

###General QA

In [ ]:
# General Questions

questions_file_path = '/content/questions/Questions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 Newton's first law of motion, also known as the law of inertia, states that an object will
remain at rest or continue moving at a constant velocity in a straight line unless acted upon by a net external force.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The force of gravity between two objects decreases with
increasing distance, following an inverse square law. This means that the force is proportional to the product of the masses of the two objects and
inversely proportional to the square of the distance between them.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Activation functions serve the purpose of introducing nonlinearity in
neural networks, allowing them to learn complex patterns and relationships in data. They introduce nonlinearity by applying an nonlinear function to
the weighted sum of inputs, outputting a new value that can then be passed through the next layer for further processing.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Attention.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Supervised learning involves learning from
labeled data, where each example includes an input and its corresponding output label. The goal is to learn a mapping from inputs to outputs based on
these examples. Unsupervised learning, on the other hand, deals with unlabeled data, meaning there is no explicit output label provided. Instead, the
algorithm looks for patterns or structures within the data itself. Common techniques include clustering, dimensionality reduction, and density
estimation.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/

 Overfitting refers to a neural network that
learns the training data too well, including its noise and irrelevant details, resulting in poor performance on new, unseen data. This can lead to
models that memorize the training data rather than generalizing to new inputs. Techniques like dropout and regularization are used to prevent
overfitting by adding randomness during training and reducing the complexity of the model.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
 The learning rate is a parameter in optimization
algorithms used to update the weights in neural networks during training. It determines the size of the weight updates at each iteration, influencing
how quickly the model converges towards the optimal solution. A high learning rate may caus

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is Newton's first law of motion?
 response:  Newton's first law of motion, also known as the law of inertia, states that an object will
remain at rest or continue moving at a constant velocity in a straight line unless acted upon by a net external force.


Q1: How does the force of gravity between two objects change with distance?
 response:  The force of gravity between two objects decreases with
increasing distance, following an inverse square law. This means that the force is proportional to the product of the masses of the two objects and
inversely proportional to the square of the distance between them.


Q2: What is the principle of conservation of energy?
 response:  The principle of conservation of energy states that energy cannot be created or
destroyed in an isolated system. It is a fundamental law of physics that applies to all forms of energy, including potential energy, kinetic energy,
thermal energy, electrical energy, and radiation energy. This principle is a di

In [ ]:
json_file_name = 'generalQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'generalQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Cosmology QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroCosmoQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 The goal of studying astrophysics and cosmology is to understand the
physical properties and evolution of the universe, including stars, galaxies, and large-scale structures, as well as the origins of the universe
itself. This field combines observations and theories to explore topics such as the nature of dark matter and dark energy, the expansion of the
universe, and the search for extraterrestrial life.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Big Bang theory is supported by various lines of evidence including the cosmic microwave
background radiation discovered by Penzias and Wilson, the abundance of light elements, the large scale structure of the universe, and the redshift of
distant galaxies among others. However, it's important to note that scientific theories are never proven to be absolutely true, but rather are
accepted based on the available evidence and the ability to make accurate predictions. The Big Bang theory continues to be the favored explanation for
the origins of the universe due to its explanatory power and consistency with observational data.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Recent developments in cosmology include the invention of the inflationary
theory by Alan Guth in the early 1980s on the theoretical side, and the first results from the Cosmic Background Explorer satellite being published in
the early 1990s on the observational side. These discoveries strengthened the expanding universe paradigm. Additionally, advancements in multi-object
fiber spectroscopy, such as the construction of the 2-degree-Field (2dF) Spectrograph and the Sloan Digital Sky Survey (SDSS), have revolutionized the
field by allowing for a larger yield of redshifts to be collected.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The observational revolution in cosmology refers to the significant
advancements in our understanding of the universe through precise observations, leading to accurate determination of cosmological parameters such as
the rate of expansion, matter content, cosmological constant, spatial curvature, and age of the universe. These discoveries provide valuable insights
into various aspects of cosmology, including the evolution of the universe, the nature of dark matter and dark energy, and the origin of structure in
the universe. This revolution has made cosmology a phenomenological science and will continue to dominate research in the new millennium.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Modern cosmological observations are explained primarily by the standard
Big Bang theory, which has been incorporated into the larger picture of cosmological inflation, forming the new standard cosmological model. These
theories provide explanations for various phenomena including the redshift-distance relation, the distribution of galaxies and their evolution, and
the large-scale structure measurements. However, it's important to remember that all physical theories are approximations of reality and may fail if
pushed too far. The standard Big Bang theory is supported by a wealth of evidence but continues to evolve within the broader context of cosmological
inflation. Other theories like plasma-redshift cosmology propose alternative explanations for certain observations, but they remain subject to ongoing
research and debate.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the goal of studying astrophysics and cosmology?
 response:  The goal of studying astrophysics and cosmology is to understand the
physical properties and evolution of the universe, including stars, galaxies, and large-scale structures, as well as the origins of the universe
itself. This field combines observations and theories to explore topics such as the nature of dark matter and dark energy, the expansion of the
universe, and the search for extraterrestrial life.


Q1: What supports the Big Bang theory?
 response:  The Big Bang theory is supported by various lines of evidence including the cosmic microwave
background radiation discovered by Penzias and Wilson, the abundance of light elements, the large scale structure of the universe, and the redshift of
distant galaxies among others. However, it's important to note that scientific theories are never proven to be absolutely true, but rather are
accepted based on the available evidence and the ability to make accurate pre

In [ ]:
json_file_name = 'AstroCosmoQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroCosmoQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Physics QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroPhysicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 The Chandrasekhar limit is a theoretical maximum mass for a
stable white dwarf star, beyond which it cannot sustain its own weight due to the degenerate pressure of its electrons. It is approximately equal to
1.43 solar masses. In astrophysics, it marks the boundary between white dwarfs and more massive objects like neutron stars or black holes.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 White dwarf stars are dense remnants of low-mass stars
that have completed their nuclear fuel burning and ejected their outer layers. They provide important insights into the later stages of stellar
evolution by demonstrating that the pressure of electron degeneracy can support a star against its own gravity, allowing it to maintain a stable
configuration despite its high density. This discovery helped resolve the mystery of how white dwarfs could exist given their extreme compactness and
the inability of normal gas pressure to support them against gravity. It also led to the realization that the maximum mass of a stable white dwarf is
determined by the Chandrasekhar limit, which marks the boundary between white dwarfs and more massive objects that collapse into other types of
compact stars like neutron stars or black holes.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astro

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


I cannot directly answer your question using the context text provided, as the text does not mention any experimental advances related to the direct
observation of gravitational waves or their significance for astrophysics. However, I can provide some background information. Gravitational waves
were first predicted by Albert Einstein's theory of general relativity in 1915. They are ripples in the fabric of spacetime caused by accelerating
masses, such as those produced during mergers of black holes or neutron stars. The Laser Interferometer Gravitational-Wave Observatory (LIGO) and
Virgo collaborations detected the first confirmed gravitational wave signals in 2015 and 2016, respectively. These detections marked a major milestone
in astrophysics and opened a new window into understanding the universe through gravitational wave observations.


Sources:
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented R

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The discovery of black holes
challenges our understanding of physics under extreme conditions by presenting us with a mathematical singularity, which is currently identified with
a black hole. However, the existence of black holes as physical objects is still debated among scientists. Some argue that they do not exist and that
the Schwarzschild singularity should be associated with another astronomical object instead. This debate highlights the ongoing efforts to understand
the behavior of matter and energy under extreme conditions predicted by general relativity.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The contemporary
challenges in theoretical physics suggested by the study of cosmological phenomena include solving initial condition problems, such as the origin of
the universe and the matter-energy budget, which cannot be answered by the standard Big Bang theory alone. These issues may be resolved or explained
by new physical principles at higher energies in the early universe. Additionally, there are many unresolved issues related to these problems, but
there might be reasons to be optimistic in the near future.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
 Based on the context provided, the
article discusses the potential application of Augmented Reality (AR) technology in advancing our exploration of cosmological phenomena. It mentions
that AR is becom

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the Chandrasekhar limit and its significance in astrophysics?
 response:  The Chandrasekhar limit is a theoretical maximum mass for a
stable white dwarf star, beyond which it cannot sustain its own weight due to the degenerate pressure of its electrons. It is approximately equal to
1.43 solar masses. In astrophysics, it marks the boundary between white dwarfs and more massive objects like neutron stars or black holes.


Q1: How do white dwarf stars contribute to our understanding of stellar evolution?
 response:  White dwarf stars are dense remnants of low-mass stars
that have completed their nuclear fuel burning and ejected their outer layers. They provide important insights into the later stages of stellar
evolution by demonstrating that the pressure of electron degeneracy can support a star against its own gravity, allowing it to maintain a stable
configuration despite its high density. This discovery helped resolve the mystery of how white dwarfs could exist given their

In [ ]:
json_file_name = 'AstroPhysicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroPhysicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

### Attention Questions

In [ ]:
# Attention Questions

questions_file_path = '/content/questions/AttQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 The Transformer model introduces the use of an attention mechanism instead of
recurrence or convolutions for drawing global dependencies between input and output.


Sources:
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Transformer model enhances training efficiency by utilizing attention
mechanisms exclusively, eliminating the need for recurrence and convolutions. This simplified architecture allows for greater parallelizability and
reduces the amount of time required to train compared to traditional RNN and Convolutional Neural Network (CNN) models.


Sources:
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Attention in Transformers is a mechanism that maps a query and a set of key-value pairs to an
output, where the query, keys, values, and output are all vectors. It allows every position in one sequence (decoder or encoder) to attend to all
positions in another sequence (input or previous layer). Multi-head attention enables the model to jointly attend to information from different
representation subspaces at different positions.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Residual dropout is applied to the output of each sub-layer before it is
added to the sub-layer input and normalized. Additionally, dropout is applied to the sums of the embeddings and the positional encodings in both the
encoder and decoder stacks. For the base model, a dropout rate of 0.1 is used.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Transformer model improved machine translation accuracy
through the use of attention mechanisms in place of recurrence and convolutions. It achieved superior quality on two machine translation tasks
compared to previous models, while being more parallelizable and requiring less time to train. Specifically, it established new state-of-the-art BLEU
scores on the WMT 2014 English-to-German and English-to-French translation tasks. Additionally, it generalized well to other tasks such as English
constituency parsing.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
 Yes, the Transformer model has been shown to generalize well to
other tasks beyond machine translation. It has been successfully applied to English constituency parsing with both large and limited training data.
Additionally, there are plans to extend the Transformer to problems involving input and output modaliti

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What innovation does the Transformer model introduce?
 response:  The Transformer model introduces the use of an attention mechanism instead of
recurrence or convolutions for drawing global dependencies between input and output.


Q1: How does the Transformer model enhance training efficiency?
 response:  The Transformer model enhances training efficiency by utilizing attention
mechanisms exclusively, eliminating the need for recurrence and convolutions. This simplified architecture allows for greater parallelizability and
reduces the amount of time required to train compared to traditional RNN and Convolutional Neural Network (CNN) models.


Q2: What are key components of the Transformer’s encoder?
 response:  The Transformer's encoder consists of a stack of N identical layers, each
containing a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. Both sub-layers have residual
connections and layer normalization. All sub-layers and 

In [ ]:
json_file_name = 'AttQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AttQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Coherent Spin Questions

In [ ]:
# Coherent Spin Questions

questions_file_path = '/content/questions/CoSpinQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 The objective of using Heisenberg exchange in quantum
computing is to manipulate and transfer quantum states between particles with high precision and minimal loss of coherence. However, the concept of
coherent spin-state transfer via Heisenberg exchange is frequently misunderstood and misrepresented within the framework of classical mechanics. It is
essential to clarify the distinct nature of quantum phenomena and rectify fundamental misunderstandings to accurately understand quantum mechanics in
modern technological applications.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Heisenberg exchange
interaction during quantum spin-state transfer is influenced by the exchange coupling constant, denoted by 𝐽. It is important to note that this
constant is a quantum mechanical property and should not be compared to a classical spring constant. Other factors include the spin vectors of the
interacting particles, 𝑆𝑖 and 𝑆𝑗. However, the misconception lies in the inclusion of non-existent gravitational interactions between nearest neighbor
pairs in the sum, denoted by ⟨𝑖,𝑗⟩. These interactions do not exist in the Heisenberg exchange interaction.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 The text discusses the importance of correctly
understanding quantum mechanics concepts, specifically regarding coherent spin-state transfer via the Heisenberg exchange interaction, to avoid
misconceptions and improve the accuracy of scientific discourse in modern technological applications, including quantum computing. It mentions the
misconception of comparing quantum state dynamics to mechanical and thermal interactions, which are irrelevant and misleading in the quantum realm.
However, there is no specific mention or discussion about quantum scalability related to coherent spin-state transfer in the given text.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Variance-based.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Heisenberg exchange interaction plays a
significant role in quantum computing applications, specifically in managing and manipulating quantum states with high precision and minimal loss of
coherence. It is a quantum mechanical effect that dictates the interaction between spins in a system, but is often misunderstood due to its
misrepresentation as a classical phenomenon. Incorrect assumptions about the nature of quantum phenomena and the use of classical terms like
temperature and mechanical forces can lead to fundamental misunderstandings that complicate the accurate understanding of quantum mechanics in modern
technological applications.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The metaphor commonly used to explain the initialization of
quantum systems is confusing it with setting up a classical mechanical system, such as loading a spring. However, this analogy is misleading as
quantum initialization involves putting a quantum system into a specific quantum state, while classical mechanical initialization refers to preparing
a physical system for motion.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/quantum.pdf
 Future research in quantum mechanics is expected to focus on
developing educational tools and resources to distinguish between classical and quantum physical theories, preventing the perpetuation of
misconceptions in scientific and technological advancements. Additionally, new fields of knowledge such as quantum information theory, quantum
thermodynamics, and the development of novel mathematical and computational tools will continue t

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the objective of using Heisenberg exchange in quantum computing?
 response:  The objective of using Heisenberg exchange in quantum
computing is to manipulate and transfer quantum states between particles with high precision and minimal loss of coherence. However, the concept of
coherent spin-state transfer via Heisenberg exchange is frequently misunderstood and misrepresented within the framework of classical mechanics. It is
essential to clarify the distinct nature of quantum phenomena and rectify fundamental misunderstandings to accurately understand quantum mechanics in
modern technological applications.


Q1: What external factors influence the Heisenberg exchange interaction during quantum spin-state transfer?
 response:  The Heisenberg exchange
interaction during quantum spin-state transfer is influenced by the exchange coupling constant, denoted by 𝐽. It is important to note that this
constant is a quantum mechanical property and should not be compared to a classical

In [ ]:
json_file_name = 'CoSpinQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'CoSpinQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Mamba Questions

In [ ]:
# Mamba Questions

questions_file_path = '/content/questions/MambaQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 Mamba achieves computational efficiency
by operating effectively under generic GPU configurations, assuming that standard GPU capabilities are sufficient to manage its computational demands
efficiently.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Mamba introduces selective state space models, which
dynamically adjust model parameters based on the output, optimizing real-time processing capabilities and adjusting computational focus dynamically.
It also extends the application of structured state space models (SSMs) to sequence data, providing a more scalable approach to sequence modeling.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Mamba's preliminary evaluations
indicate that its effectiveness on longer sequences and more complex datasets needs thorough benchmarking against established models like
Transformers. Additionally, the open-sourcing strategy and potential limitations on modifications may impact its adoption and further development
across diverse applications.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Dynamic parameter adjustment is a key
feature of Mamba's selective state space models, allowing for optimized real-time processing capabilities and dynamic computation focus, shifting from
input-dependent parameter adjustments seen in traditional models.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Mamba.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Mamba addresses the inefficiencies
of Transformers in moderate to long sequence processing by leveraging structured state space models (SSMs) and integrating RNN-like and CNN-like
layers while reintroducing MLP blocks. It operates effectively under standard GPU configurations and reduces dependency on traditional attention
mechanisms.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
 The design of Mamba, which
utilizes structured state space models (SSMs) and merges RNN-like and CNN-like layers, allows it to process various data types more efficiently
compared to traditional methods. However, its effectiveness on long sequences and complex datasets, especially when compared to established models
like Transformers, has not been extensively evaluated yet. Additionally, the restricted release strategy for Mamba may limit its modification and use
within the broader AI community, potentially impacti

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: How does Mamba achieve computational efficiency without specialized hardware optimizations?
 response:  Mamba achieves computational efficiency
by operating effectively under generic GPU configurations, assuming that standard GPU capabilities are sufficient to manage its computational demands
efficiently.


Q1: What innovations does Mamba introduce to manage long sequence data processing?
 response:  Mamba introduces selective state space models, which
dynamically adjust model parameters based on the output, optimizing real-time processing capabilities and adjusting computational focus dynamically.
It also extends the application of structured state space models (SSMs) to sequence data, providing a more scalable approach to sequence modeling.


Q2: How does the selective state space model in Mamba differ from traditional attention mechanisms in handling sequence data?
 response:  The main
difference between selective state space models in Mamba and traditional attention mechanisms 

In [ ]:
json_file_name = 'MambaQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'MambaQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Parametric Magnon Questions

In [ ]:
# Parametric Magnon Questions

questions_file_path = '/content/questions/ParametricMagnonQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 The typical materials used to construct
quantum transducers in hybrid quantum systems are yttrium iron garnet (YIG) and nitrogen-vacancy (NV) defects in diamond. However, the approach
presented in the given work uses wafer-compatible materials to engineer a hybrid transducer that exploits magnon nonlinearities in a magnetic
microdisc to address quantum spin defects in silicon carbide.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Magnon nonlinearities enhance quantum transduction by allowing selective
tuning of the spin-magnet coupling "on" and "off," providing protection against resonant magnon noise-induced decoherence, and enabling tunable
transduction between distinct physical components. This contrasts with the constant dipolar coupling between spin centers and linear magnons in
traditional systems.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Parametric magnonics introduces a new approach
in quantum transducers by using nonlinear magnonics to engineer a hybrid transducer that exploits magnon nonlinearities in a magnetic microdisc to
address quantum spin defects. This approach offers unique transduction behavior compared to traditional linear magnon transducers.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Introducing nonlinear magnonics to quantum computing
systems provides alternative perspectives for engineering quantum interfaces to spin qubits and motivates further research into uncovering the
interesting phenomena lying at the intersection of nonlinear magnonics and quantum systems. It offers the potential for enhanced coupling strengths
and cooperativities through the generation of squeezed magnon states.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Magnon nonlinearities in quantum systems could
potentially lead to enhanced coupling strengths and cooperativities in hybrid quantum systems, exponentially enhancing their performance. These
nonclassical states have the potential to bring systems into the strong coupling regime and reduce damping processes at cryogenic temperatures. Future
applications could include quantum computing, where magnon nonlinearities could help expand the quantum engineer's toolbox and improve the
functionality of existing quantum systems.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
 Recent research in quantum magnonics
provides alternative perspectives for engineering quantum interfaces to spin qubits by exploring nonlinear magnonic systems. These systems offer
unique functionalities due to their wide range of magnon interactions and intrinsic nonline

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What materials are typically used to construct quantum transducers in hybrid quantum systems?
 response:  The typical materials used to construct
quantum transducers in hybrid quantum systems are yttrium iron garnet (YIG) and nitrogen-vacancy (NV) defects in diamond. However, the approach
presented in the given work uses wafer-compatible materials to engineer a hybrid transducer that exploits magnon nonlinearities in a magnetic
microdisc to address quantum spin defects in silicon carbide.


Q1: How do magnon nonlinearities enhance quantum transduction?
 response:  Magnon nonlinearities enhance quantum transduction by allowing selective
tuning of the spin-magnet coupling "on" and "off," providing protection against resonant magnon noise-induced decoherence, and enabling tunable
transduction between distinct physical components. This contrasts with the constant dipolar coupling between spin centers and linear magnons in
traditional systems.


Q2: What techniques are used to minimize 

In [ ]:
json_file_name = 'ParametricMagnonQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'ParametricMagnonQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Quantum Mechanics Questions

In [ ]:
# Quantum Mechanics Questions

questions_file_path = '/content/questions/QuantumMechanicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 Quantum mechanics is significant in modern technology because it
has led to the creation of new fields of knowledge, such as quantum information theory and quantum thermodynamics, and the development of novel
mathematical and computational tools applicable to various domains. Additionally, an improved understanding of the resource power of quantum phenomena
has triggered a technological overhaul, resulting in the development of superfast quantum computers, unbreakable quantum cryptography, ultrasensitive
quantum sensors, and other disruptive technologies. These innovations have the potential to rival the three major industrial revolutions of the last
century.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Quantum mechanics has led to the creation of new fields of study such as
quantum information theory, quantum thermodynamics, and the development of novel mathematical and computational tools applicable to other domains
including condensed matter physics, statistical mechanics, and cosmology.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 There is considerable debate regarding the
interpretation of the foundations of quantum mechanics, specifically concerning issues such as quantum measurement, quantum randomness, non-locality,
particle indistinguishability, causality, and the nature of time. These topics continue to be explored at the forefront of quantum research.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Exploring quantum mechanics contributes to theoretical
physics by shedding light on the physical meaning of fundamental quantum principles and pushing the boundaries of the quantum description of the
world. It leads to advancements in various areas of quantum research, including quantum measurement, quantum randomness, non-locality, particle
indistinguishability, causality, and the nature of time. These discoveries have significant implications for both basic science and technology.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Decades of experimental verification and control of quantum
systems have proven detractors of quantum mechanics wrong. Examples include experiments on superposition, wave-particle duality, uncertainty
principle, entanglement, and non-locality. (References: 3-7)


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
 Anticipated future developments in quantum technology include the
creation of new fields of knowledge such as quantum information theory and quantum thermodynamics, the development of novel mathematical and
computational tools, and the achievement of concrete impacts through the improvement of understanding of quantum phenomena's resource power, leading
to technological overhauls. Exciting prospects include superfast quantum computers, unbreakable quantum cryptography, ultrasensitive quantum sensors,
exploration of the interplay of quantum mechanics with black-hole physics and t

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the significance of quantum mechanics in modern technology?
 response:  Quantum mechanics is significant in modern technology because it
has led to the creation of new fields of knowledge, such as quantum information theory and quantum thermodynamics, and the development of novel
mathematical and computational tools applicable to various domains. Additionally, an improved understanding of the resource power of quantum phenomena
has triggered a technological overhaul, resulting in the development of superfast quantum computers, unbreakable quantum cryptography, ultrasensitive
quantum sensors, and other disruptive technologies. These innovations have the potential to rival the three major industrial revolutions of the last
century.


Q1: How has quantum mechanics influenced new fields of study?
 response:  Quantum mechanics has led to the creation of new fields of study such as
quantum information theory, quantum thermodynamics, and the development of novel mathematical and c

In [ ]:
json_file_name = 'QuantumMechanicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'Quantum MechanicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Qubit Teleportation Questions

In [ ]:
# Qubit Teleportation Questions

questions_file_path = '/content/questions/QubitTeleportationQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 The main components used in quantum networks for
teleportation include optically connected nodes, a teleporter in the form of a pre-shared entangled state, and a memory qubit for storing the
teleported qubit state. Additionally, there are key innovations such as improved qubit readout procedures, active memory qubit protection during
entanglement generation, and tailored heralding to reduce remote entanglement infidelities.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Entanglement is established between distant nodes in a
quantum network through a process called entanglement swapping, mediated by a third node. This involves first generating entanglement between
neighboring nodes using a single-photon protocol, then performing an entanglement swapping protocol on the middle node to establish entanglement
between the distant nodes.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Memory qubits play a crucial role in the
process of quantum teleportation in a network by serving as a storage medium for the entangled state between neighboring nodes. During the
teleportation protocol, the input qubit state is prepared on one of the memory qubits, which is then teleported to another node through the exchange
of classical information and local operations. Additionally, memory qubits must be reliably preserved during the teleportation process to maintain the
entangled state between nodes. This is achieved through techniques such as error correction and active coherence protection from the spin bath.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Potential future applications of quantum
teleportation in quantum networks include executing and testing quantum algorithms and protocols through platform-independent control software,
improving phase stabilization, and extending current schemes for use in deployed fiber networks. Additionally, quantum teleportation can enable more
complex protocols and applications.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Real-time feed-forward enables unconditional quantum
teleportation by allowing state transfer each time a qubit state is inserted into the teleporter. However, it requires perfect entanglement, Bell
state measurements, and coherence times for enabling real-time feed-forward, which has prevented its realization beyond directly connected stationary
network nodes so far. The presence of various error sources such as imperfect Bell states, dephasing, depolarizing noise, and readout errors also
affect the teleportation process.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
 The technical
advancements required to realize efficient quantum teleportation between non-neighbouring nodes include improvements in the generation and maintenance
of remote entanglement, joint qubit readout, and coherence times. Additionally, real-time rej

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What are the main components used in quantum networks for teleportation?
 response:  The main components used in quantum networks for
teleportation include optically connected nodes, a teleporter in the form of a pre-shared entangled state, and a memory qubit for storing the
teleported qubit state. Additionally, there are key innovations such as improved qubit readout procedures, active memory qubit protection during
entanglement generation, and tailored heralding to reduce remote entanglement infidelities.


Q1: How is entanglement established between distant nodes in a quantum network?
 response:  Entanglement is established between distant nodes in a
quantum network through a process called entanglement swapping, mediated by a third node. This involves first generating entanglement between
neighboring nodes using a single-photon protocol, then performing an entanglement swapping protocol on the middle node to establish entanglement
between the distant nodes.


Q2: What role does

In [ ]:
json_file_name = 'QubitTeleportationQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QubitTeleportationQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Variance Based Questions

In [ ]:
# Variance Based Questions

questions_file_path = '/content/questions/VarianceQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

 The focus of variance-based sensitivity analysis in
quantum memory is to shed light on the sensitivity of an individual quantum memory implementation to device-specific fluctuations and drift, as well
as on the intrinsic sensitivity of different physical quantum memory protocols. It provides a complete picture of the system performance landscape
around a central point of input parameters and allows for identification of which input parameters are most sensitive globally. Additionally, it
probes whether correlations exist between parameters, which can be leveraged to allow for acceptable system performance at nonoptimal parameter
values.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Λ-type quantum memory refers to a specific type of quantum memory used in optics,
particularly in the context of atomic ensembles. It gets its name from the shape of the energy level scheme involved in the memory process, which
forms a lambda ("Λ") configuration.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Memory sensitivity analysis is important because it helps determine how a
system's performance, such as memory efficiency, is affected by fluctuations in its input parameters while keeping internal system parameters fixed.
Low sensitivity indicates a larger acceptable region of control field phase space where acceptable memory performance can be achieved, making the
memory robust to experimental drift. The tools developed in this work are also applicable to other level systems and related techniques beyond
resonant Lambda1-type memory protocols.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The text discusses the use of variance-based sensitivity
analysis for studying memory sensitivity in quantum systems. Specifically, it calculates the sensitivity index for different control field parameters
such as delay and pulse duration using the Sobol' method. It also mentions that other types of sensitivity, like fidelity sensitivity, can be analyzed
using the same framework but may have experimental considerations beyond those discussed in the text.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Gaussian control field relates to memory performance through
its optimization for maximizing memory efficiency. Its parameters, such as pulse area, delay, and duration, define a temporal envelope that shapes the
interaction between the memory and the control field. Memory efficiency fluctuations can arise due to variations in these control field parameters, as
well as correlations between them. The sensitivity of memory performance to these control field parameters can be quantified using methods like one-
at-a-time analysis or Sobol' sensitivity indices. Additionally, the overlap felidity between optimal control fields defines how similar they are, and
regions with less overlap felidity, such as the absorb-then-transfer protocol, are more sensitive to fluctuations in memory parameters.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
 The findin

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the focus of variance-based sensitivity analysis in quantum memory?
 response:  The focus of variance-based sensitivity analysis in
quantum memory is to shed light on the sensitivity of an individual quantum memory implementation to device-specific fluctuations and drift, as well
as on the intrinsic sensitivity of different physical quantum memory protocols. It provides a complete picture of the system performance landscape
around a central point of input parameters and allows for identification of which input parameters are most sensitive globally. Additionally, it
probes whether correlations exist between parameters, which can be leveraged to allow for acceptable system performance at nonoptimal parameter
values.


Q1: What does Λ-type quantum memory refer to?
 response:  Λ-type quantum memory refers to a specific type of quantum memory used in optics,
particularly in the context of atomic ensembles. It gets its name from the shape of the energy level scheme involved in t

In [ ]:
json_file_name = 'VarianceQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'VarianceQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)